# Create Permits, Violations, and TaxParcels File with MGO Data and AS 400 Data.



<b> TO DO: </b>
<ul> <li>get values out of the year column to create multiple DFs using those years.</li>
<li>also can create a dictionary with those years in there.</li>
<li>use dask to create a lot of DFs for processing.</li>
<li>Add GeoSpatial Data to All Permits for Historical Permits.</li>
<li>Remove Permits from the DB and upload the new Permits from MGO and AS400 Data.</li>
</ul>

## Import Modules and Declare Global Variables for geoprocessing

In [ ]:
#Use arcNew environment with this notebook.
#Current Location: C:\ProgramData\Anaconda3\envs\arcNew\python.exe
import arcpy as arc
import os
from shutil import copy2
import pandas as pd
import numpy as np
from arcgis import features
from datetime import datetime as dt
from pathlib import Path

In [ ]:
arc.env.overwriteOutput = True
arc.env.outputZFlag = 'Disabled' #To remove z data from parcel fabric due to it being a polygonZ
arc.env.outputMFlag = 'Disabled'
arc.env.qualifiedFieldNames = False
now = dt.now()
mStr = now.strftime('%m%Y')
dStr = now.strftime('%m_%d')
uPath = Path.home()
locFolders = ['Processing', 'Review']
if uPath.exists():
    for x in locFolders:
        a = Path(uPath / 'GIS' / x)
        if a.exists():
            print(f'{a} already exists.')
        else:
            a.mkdir(parents=True)
            print(f'{a} has been created.')
else:
    pass

gisPath = uPath / 'GIS'
lPath = [f for f in gisPath.glob('*')]
netDir = Path(r'\\kcdp-1\KCGIS\MasterGISFiles\Ben')
netDB = netDir / 'GISPro' / 'SDE Connections'
# print(netDB)

### Only use with notebook

In [ ]:
pd.options.display.max_columns = 40

## For use when in production.

### Create Folders for Permits Review Data

In [ ]:
#Create Folders for Permits Data
pFolder = [f for f in lPath if f.name == 'Processing'][0]
pProcessing = pFolder / 'Permits' / f'{dStr}'
if pProcessing.exists() == True:
    print(f'{pProcessing} already exist.')
else:
    pProcessing.mkdir(parents=True)
    print(f'Created {pProcessing}.')

pFR = [f for f in lPath if f.name == 'Review'][0]
pReview = pFR / 'Permits' / f'{dStr}'
if pReview.exists() == True:
    print(f'{pReview} already exist')
else:
    pReview.mkdir(parents=True)
    print(f'Created {pReview}')

### Create New File GeoDatabase and corresponding Feature Datasets.

In [ ]:
iE = netDB / 'MAPPINGADMIN.sde' / 'PROD.MAPPINGADMIN.ParcelEditing'
sr = arc.Describe(f'{iE}').spatialReference
outGDB = gisPath / pFolder / f'Data_{mStr}.gdb'
# dsA = gisPath / pFolder / f'{outGDB}.gdb'
locGDB = outGDB / f'Daily_{dStr}'
if arc.Exists(f'{outGDB}'):
    print("GDB already exists.")
else:
    arc.CreateFileGDB_management(f'{pFolder}', f'{outGDB.name}')
    print(f'Created File GeoDatabase at {outGDB.parent}')

time.sleep(2)

if arc.Exists(f'{locGDB}'):
    print(f'{locGDB.name} already exists')
else:
    arc.CreateFeatureDataset_management(f'{locGDB.parent}', f'{locGDB.name}', sr)
    print(f'{locGDB.name} Dataset has been created')

## Create Parcel Fabric DataFrame and Points FC

### Does not work with Parcel Fabric due to bug with 10.5.1

In [ ]:
dbAdd = 'PROD.ADDRESSINGADMIN.Addressing'
dbSufP = 'PROD.GISADMIN.Suffix_Parcels'
dbMPTab = 'PROD.gisadmin.MGOPermits'
dbAPTab = 'PROD.gisadmin.All_Permits'

In [ ]:
#create paths to the data instead of using env.workspace makes it so that I can use arc Walk
arc.env.workspace = f'{iE.parent}'

fcList = [dbAdd, dbSufP]
tList = [dbMPTab, dbAPTab]
cParcels = []
cTables = []
for dirpath, dirnames, filenames in arc.da.Walk():
    for f in filenames:
        if f in fcList:
            a  = f.split('.')[-1]
            b = f'{a}_{dStr}'
            if arc.Exists(f'{locGDB / b}'):
                cParcels.append(locGDB.joinpath(b))
                print(f'{b} already exists')
            else:
                arc.FeatureClassToFeatureClass_conversion(f, f'{locGDB}', f'{b}')
                cParcels.append(locGDB.joinpath(b))
                print (f'{b} has been copied')
for dirpath, dirnames, filenames in arc.da.Walk():
    for f in filenames:
        if f in tList:
            a  = f.split('.')[-1]
            b = f'{a}_{dStr}' #Tables need to have a date string since they won't be under the dataset
            # c = locGDB.parent
            if arc.Exists(f'{outGDB / b}'):
                cTables.append(outGDB.joinpath(b))
                print(f'{b} already exists')
            else:
                arc.TableToTable_conversion(f, f'{outGDB}', f'{b}')
                cTables.append(outGDB.joinpath(b))
                print (f'{b} has been copied')
# print(cParcels)
# print(cTables)

### Copy Parcel Fabric to local folder

In [ ]:
#currently using env.workspace to move parcelfabric needs to be changed to use path
pfParcels = 'PROD.MAPPINGADMIN.ParcelFabric_Parcels'
exp = "TYPE = 7 AND Historical = 0"
a = pfParcels.split('.')[-1]
b = f'{a}_{dStr}'
# print(f'{iEPath[1]}')
if arc.Exists(f'{locGDB / b}'):
    cParcels.append(locGDB.joinpath(b))
    print(f'{b} already exists')
else:
    print(f'Copying Parcel Fabric to {locGDB}')
    #print(a)
    # g = f'{f}'
    #print(g)
    arc.FeatureClassToFeatureClass_conversion(pfParcels, f'{locGDB}', b, where_clause=exp)
    cParcels.append(locGDB.joinpath(b))
    print(f'Finished Copying {b} to {locGDB}')

<b>Create points out of Parcel Fabric to be used with Permits.</b>

In [ ]:
#Create Points out of polygons in ParcelFabric
pfFC = [f for f in cParcels if f.name.startswith('ParcelFabric') == True][0]
# print(pfFC)
pfPoints = locGDB / f'pfPoints_{dStr}'
# print(pfPoints)
if arc.Exists(f'{pfPoints}'):
    print(f'{pfPoints.name} already exists.')
else:
    arc.FeatureToPoint_management(str(pfFC), f'{pfPoints}', "INSIDE")
    print(f'{pfPoints.name} has been created.')

<b> Get List of Fields from Parcel Fabric, then create DataFrame for processing </b>

In [ ]:
# pfFields = [f.name for f in arc.ListFields(pfFC)]
# print(pfFields)
pf_df = features.GeoAccessor.from_featureclass(pfPoints, fields=['Name'])
print('Parcel Fabric DataFrame created.')
# pf_df.shape

Create DataFrame from Parcel Fabric Polygons

In [ ]:
# pfFC = [f for f in cParcels if f.name.startswith('ParcelFabric') == True][0]
ppf_df = features.GeoAccessor.from_featureclass(pfFC, fields=['Name'])
print('Parcel Fabric DataFrame created.')

<b> Create list of columns in DataFrame to check for missing columns. </b>

In [ ]:
# pfC = pf_df.columns.tolist()
# print(pfC)

## MGO Permits DF Creation and Data Curation.

<b> Create DataFrame from MGO Table </b>

In [ ]:
mTab = [f for f in cTables if f.name.startswith('MGOPermits') == True][0]
# mTable = str(tLoc / mgoT)
# print(mTable)
df_mgo = features.GeoAccessor.from_table(mTab, skip_nulls= False)
print(f'{mTab.name} has been created')
# df_mgo.shape

<b> Create DataFrame for Permits Description to use for merge with MGO Permits. </b>

In [ ]:
# move to local GIS Folder for Processing
appCsv = netDir / 'ParcelWithSuffix' / 'Permits_Desc.csv'
locCsv = pProcessing / 'Permits_Desc.csv'
# print(locCsv)
copy2(appCsv, locCsv)
print(f'Copied {locCsv.name} to {locCsv.parent}')

time.sleep(2)

p_desc = pd.read_csv(locCsv, keep_default_na=False)
print('Created Permits Description DataFrame')
# p_desc

In [ ]:
#columns and values have been fixed on the spreadsheet. Does not to be automated due to it never changing.
#not for us in production
# cFix = p_desc.columns.tolist()
# pList = [f.strip() for f in cFix]
# print(pList)

# cpFix = dict(zip(cFix, pList))
# print(cpFix)

# p_desc = p_desc.rename(columns=cpFix)
# # p_desc


Edit the MGO Dates and format them to match AS400 Data.

In [ ]:
# Upper Case all the column names
df_mgo = df_mgo.rename(columns=lambda x: x.upper())

In [ ]:
def fdate(field):
    from datetime import date
    po = date(2021, 1, 1)
    if field >= po:
        return field
    else:
        return ''


In [ ]:
df_mgo.CODATE = df_mgo.CODATE.apply(fdate)
# df_mgo['CODATE'] = pd.to_datetime(df_mgo.CODATE)
# df_mgo.drop(columns='CODate1', inplace=True)

In [ ]:
# df_mgo.CODATE.fillna('', inplace=True)

In [ ]:
df_mgo.ISSUEDATE = df_mgo.ISSUEDATE.apply(fdate)
# df_mgo['ISSUEDATE'] = pd.to_datetime(df_mgo.CODATE)

In [ ]:
df_mgo.APPDATE = df_mgo.APPDATE.astype(str)
df_mgo.ISSUEDATE = df_mgo.ISSUEDATE.astype(str)
df_mgo.CODATE= df_mgo.CODATE.astype(str)

In [ ]:
# df_mgo['CODATE'] = df_mgo['CODATE'].apply(date)
# df_mgo.CODATE.unique()

In [ ]:
df_mgo.ISSUEDATE.replace('NaT', '', inplace=True)

In [ ]:
def date(field):
    if field != '':
        sp = field.split('-')
        return '{}/{}/{}'.format(sp[1], sp[-1], sp[0])
    else:
        return ''

In [ ]:
df_mgo.APPDATE = df_mgo.APPDATE.apply(date)
df_mgo.CODATE = df_mgo.CODATE.apply(date)
df_mgo.ISSUEDATE = df_mgo.ISSUEDATE.apply(date)

In [ ]:
df_mgo.drop(columns='OBJECTID', inplace=True)
# df_mgo

In [ ]:
# df_mgo['PERMITYEAR'] = df_mgo['PERMITNUMBER'].apply(lambda x: x.split('-')[0])
# df_mgo

In [ ]:
df_mgo = df_mgo.rename(columns={'APPTYPE' : 'MGOTYPE', 'PERMITSTATUS': 'MGOSTATUS'})
# df_mgo.columns

In [ ]:
df_mgo['TYPE'] = df_mgo['PERMITNUMBER'].apply(lambda x: x.split('-')[-1])
# df_mgo

In [ ]:
# To find the MGO permits that are not in 2021
# df_mgo_drop = df_mgo.loc[df_mgo['PERMITYEAR'] != "2021", :]
# df_mgo_drop
#drop the permits not in 2021
df_mgo = df_mgo.loc[df_mgo['PERMITYEAR'] == "2021", :]
# df_mgo
# df_mgo_drop.to_csv
# df_mgo = df_mgo.loc[df_mgo['PERMITYEAR'] == '2021']
# df_mgo

In [ ]:
# mGODict = df_mgo.to_dict('records')
# print(mGODict)
mCSV = pReview / 'mgoPerms.csv'
df_mgo.to_csv(mCSV, index=False)

Export for Review later to see the outer merge which will return all the values and which tables they are located in.<br>
Merge to get status and Description from Permit Description DataFrame

In [ ]:
#Export for Review later to see the outer merge which will return all the values and which tables they are located in. 
df_mgo_desc_o = df_mgo.merge(p_desc[['TYPE', 'DESCRIPTION']], how='outer', on='TYPE', indicator= True) #merge to create description column
tEr = pReview / 'Type_Error.csv'
df_mgo_desc_o.loc[df_mgo_desc_o._merge == 'left_only', :].to_csv(tEr, index=False) # export csv if any errors
df_mgo_desc_o = df_mgo_desc_o.drop(columns='_merge')

#merge to get status from Permit Description DataFrame
df_mgo_mis = df_mgo_desc_o.merge(p_desc[['MGO_DESC', 'MGO_INIT']], how='outer',
left_on='MGOSTATUS', right_on='MGO_DESC', indicator=True) #merge to create status column
mgoM = pReview / 'Status_Error.csv'
df_mgo_mis.loc[df_mgo_mis._merge == 'left_only'].to_csv(mgoM, index=False)
mgo_df = df_mgo_mis.loc[df_mgo_mis._merge == 'both']
# df_mgo_mis[df_mgo_mis._merge == 'both']
# merMTable = join(locFolder, f'MGOPermitsMerge_{dStr}.csv')
#not for production, used to find null values
# # df_mgo_mis.loc[df_mgo_mis['_merge'] != 'both', :]
# # df_mgo_mis.loc[df_mgo_mis['_merge'] == 'left_only', :]
# df_mgo_mis.to_csv(merMTable, index=False)
# del df_mgo_desc_o
print("Exported MGO Permits Merge Table.")

In [ ]:
# df_mgo_mis[df_mgo_mis.PARCELID == '7-00-10302-01-4300-00001']
# df_mgo_mis.loc[(df_mgo_mis.PARCELID.duplicated() == True) & (df_mgo_mis._merge == 'both'), :].groupby(by='PARCELID')
# print(dList)

## All_Permits DF Creation and Data Curation

<b> Create DataFrame from All_Permits table on Local GDB after exporting from ESRIDB. If arc.env.workspace is changed need to change aPerms to full path.</b>

In [ ]:
aTab = [f for f in cTables if f.name.startswith('All_Permits') == True][0]
# mTable = str(tLoc / mgoT)
# print(mTable)
ap_df = features.GeoAccessor.from_table(aTab, skip_nulls= False)
print(f'{aTab.name} has been created')
# ap_df.shape

In [ ]:
#Remove whitespace and make it upper
ap_df.rename(columns=lambda x: x.strip().upper(), inplace=True)
ap_df.drop(columns='OBJECTID', inplace=True)

Fix a blank entry from PRIDE.

In [ ]:
ap_df['PARCELID'] = ap_df['PARCELID'].apply(lambda x: '7-00-12900-02-5200-00001' if x.strip() == '-00-12900-02-5200-00001' else x)

Create Date columns for sorting by date later.

In [ ]:
def newyear(field):
    if int(field) > 80 and int(field) < 100:
        return "19" + str(field)
    elif int(field) >= 0 and int(field) <=9:
        return "20" + str(field)
    elif int(field) > 9 and int(field) < 80:
        return "20" + str(field)
ap_df['YEAR'] = ap_df['YEAR'].apply(newyear)

In [ ]:
# df_ap['YEAR_FORM'] = df_ap['YEAR'].apply(lambda x: f'{x[0:3]}{x[-1]}') For Correction of mistake

In [ ]:
ap_df['PERMITNUMBER'] = ap_df[['YEAR', 'PERMITNUMBER', 'TYPE']].apply(lambda x: f'{x[0]}-{x[1]}-{x[2]}', axis=1)
ap_df = ap_df[ap_df.YEAR != '2021']

In [ ]:
ap_df.drop(columns='DESCRIPTION', inplace=True)
ap_df_desc = ap_df.merge(p_desc[['TYPE', 'DESCRIPTION']], on='TYPE', how='outer', indicator= True)

In [ ]:
ap_df_b = ap_df_desc[ap_df_desc._merge == 'both']

In [ ]:
ap_df_b.drop(columns=['PERMITDESC', '_merge'], inplace= True)

In [ ]:
ap_df_b.reset_index(drop=True, inplace=True)

In [ ]:
# ap_m.loc[ap_m.SHAPE.isnull(), :]
ap_m.spatial.set_geometry('SHAPE', sr=26957)

In [ ]:
lal = gisPath / 'HPermits.csv'
ap_m.to_csv(lal)

In [ ]:
import geopandas as gpd
papa = pd.read_csv(lal)
p2p = gpd.GeoDataFrame(papa, geometry='SHAPE', crs=26957)

In [ ]:
pmdf = features.GeoAccessor.from_df(ap_m, sr=26957, geometry_column='SHAPE')

In [ ]:
#Create Feature Class for Historical Data, contains some geospatial information.
hPFC = locGDB / f'Historical_Permits_{dStr}'
ap_m = pf_df.merge(ap_df_b, how='outer', left_on='Name', right_on='PARCELID', indicator=True)
ap_m = ap_m[ap_m._merge != 'left_only']
ap_m.drop(columns='_merge', inplace=True)
ap_m.fillna('', inplace=True)

a = ap_m.spatial.to_featureset()
a.save(f'{outGDB}', 'jo')

# ap_m.spatial.to_featureclass(f'{hPFC}', sanitize_columns=False)

print(f'Exported Historical Permits to {hPFC}')

In [ ]:
ap_df_b.columns

In [ ]:
ad = features.GeoAccessor.from_df(ap_m, geometry_column='SHAPE')
ad

In [ ]:
ap_m.iloc[0]

In [ ]:
a = ap_m.spatial.to_featureset()

In [ ]:
a

In [ ]:
a.save(f'{hPFC.parent}', f'{hPFC.name}')

In [ ]:
dup_ap = ap_df_b.sort_values(by='YEAR')
dup_ap = ap_df_b.drop_duplicates(subset=['YEAR', 'PARCELID', 'TYPE'], keep='last', ignore_index=True)

In [ ]:
#Drop RJ and HD status from the permits
dup_ap = dup_ap.loc[(dup_ap.STATUS != 'RJ') | (dup_ap.STATUS != 'HD')]
# dup_ap.STATUS.value_counts()

In [ ]:
dup_ap = dup_ap.merge(p_desc[['AS_DESC', 'AS_INIT']], how='left', left_on='STATUS', right_on='AS_INIT')

In [ ]:
aPerm = pReview / 'AS_Perms.csv'
dup_ap.to_csv(aPerm, index=False)

## Merge MGO Permits and AS400 Permits into one table and get geospatial information from Parcel Fabric Points.

In [ ]:
#Drop and rename columns from AS400 and MGO DataFrames
mgo_df.drop(columns='_merge', inplace=True)
dup_ap.drop(columns='AS_INIT', inplace=True)
dup_ap.rename(columns={'AS_DESC' : 'STATDESC'}, inplace=True)

#Remove Parcel #s that are in AS400 Data (dup_ap)
pmf = mgo_df[~mgo_df.PARCELID.isin(dup_ap.PARCELID)]
pmf.rename(columns={'AS_DESC' : 'STATDESC'}, inplace=True)

In [ ]:
#Merge the pmf with dup_ap to get a list of all permits from MGO and AS400
perm_df = pmf.merge(dup_ap, on='PARCELID', how='outer', indicator=True)

In [ ]:
#concat the columns into one.
perm_df['TYPE'] = perm_df.TYPE_x.str.cat(perm_df.TYPE_y, na_rep='')
perm_df['PERMITNUMBER'] = perm_df.PERMITNUMBER_x.str.cat(perm_df.PERMITNUMBER_y, na_rep='')
perm_df['DESCRIPTION'] = perm_df.DESCRIPTION_x.str.cat(perm_df.DESCRIPTION_y, na_rep='')
perm_df['APPDATE'] = perm_df.APPDATE_x.str.cat(perm_df.APPDATE_y, na_rep='')
perm_df['CODATE'] = perm_df.CODATE_x.str.cat(perm_df.CODATE_y, na_rep='')
perm_df['ISSUED'] = perm_df.ISSUEDATE.str.cat(perm_df.DATE, na_rep='')
perm_df['STATUSDESC'] = perm_df.STATDESC.str.cat(perm_df.MGO_DESC, na_rep='')
perm_df['PERMSTATUS'] = perm_df.STATUS.str.cat(perm_df.MGO_INIT, na_rep='')
perm_df['PERMYEAR'] = perm_df.YEAR.str.cat(perm_df.PERMITYEAR, na_rep='')

In [ ]:
#Columns to keep for the Feature Class on ESRIDB and the REST Server
pmCols = ['PARCELID', 'TYPE', 'PERMSTATUS', 'ISSUED', 'PERMYEAR', 'PERMITNUMBER', 'DESCRIPTION', 'SHAPE', 'CODATE', 'APPDATE']
pmFields = [f.name for f in arc.ListFields('PROD.GISADMIN.Permits')][1:]
pmFields.remove('PermitNumb')
kpmCols = dict(zip(pmCols, pmFields))
kpmCols['STATUSDESC'] = 'StatusDesc'
pmCols.append('STATUSDESC')

In [ ]:
pm_df = perm_df.drop(columns=[f for f in perm_df.columns if f not in pmCols])
#Rename Columns
pm_df.rename(columns=kpmCols, inplace=True)

In [ ]:
pm_df.AppDate = pd.to_datetime(pm_df.AppDate, format='%m/%d/%Y', errors='coerce')
pm_df.IssueDate = pd.to_datetime(pm_df.IssueDate, format='%m/%d/%Y', errors='coerce')
pm_df.PermitYear = pd.to_datetime(pm_df.PermitYear, format='%Y', errors='coerce')
pm_df.CODate = pd.to_datetime(pm_df.CODate, format='%m/%d/%Y', errors='coerce')

In [ ]:
#Prepare Parcel Fabric Points and Merge with Permit Data
pf_df.Name = pf_df.Name.apply(lambda x: x.strip())
pfm_df = pm_df.merge(pf_df, on='Name', how='outer', indicator=True)

In [ ]:
#s_pm is all the permits with suffix and t_pm is parcels that aren't in Parcel Fabric
sOut = pReview / 'Permits_Suffix.csv'
tOut = pReview / 'Permits_Parcels_NM.csv' #NM = No Match
s_pm = pfm_df[(pfm_df._merge == 'left_only') & (~pfm_df.Name.str.contains('-00001', na=False))]
t_pm = pfm_df[(pfm_df._merge == 'left_only') & (pfm_df.Name.str.contains('-00001', na=False))]
s_pm.to_csv(sOut, index=False)
t_pm.to_csv(tOut, index=False)

print(f'Exported {sOut.name} and {tOut.name}')

In [ ]:
#Export Permits to GDB and append to Permits on the DB. Not all Permits contain GeoSpatial Information. 
perm_db = locGDB / 'Permits_DB'

db_pm = pfm_df[pfm_df._merge != 'right_only']
db_pm.drop(columns='_merge', inplace=True)
# db_pm.spatial.to_featureclass(perm_db, sanitize_columns=False)

### Create DEMO Permits Feature Class

Create AS400 Demo Permits DataFrame

In [ ]:
#Export Demo Permits for Review
dCsv = pReview / 'DemoPermits.csv'
df_demo = pm_df.query('StructureType == "DEMO" | StructureType == "DEMS" | StructureType == "MHDM"')
# df_demo.reset_index(drop=True, inplace=True)
df_demo.to_csv(dCsv, index=False)

In [ ]:
df_demo

In [ ]:
#Merge Parcel Fabric Polygons and df_demo Data
pf = features.GeoAccessor.from_featureclass(f'{pfFC}', fields=['Name'])
pf.Name = pf.Name.apply(lambda x: x.strip())
demo_fc = pf.merge(df_demo, left_on='Name', right_on='Name', how='outer', indicator=True)

In [ ]:
# demo_fc.loc[(demo_fc._merge == 'left_only') & (demo_fc.PARCELID.str.contains('-00001', na=False))]

In [ ]:
# export Parcel Names that don't match
dLo = pReview / 'Missing_Demo_P.csv'
demo_fc[demo_fc._merge == 'left_only'].to_csv(dLo, index=False)

In [ ]:
demo_fc = demo_fc[demo_fc._merge == 'both']

In [ ]:
demo_fc.columns

In [ ]:
#Columns to keep for the Feature Class on ESRIDB for employees only
ddCols = ['PARCELID', 'TYPE', 'PERMITNUMBER', 'DESCRIPTION', 'APPDATE', 'CODATE', 'ISSUED', 'STATUSDESC', 'PERMSTATUS', 'PERMYEAR', 'SHAPE']
db_demo = demo_fc.drop(columns=[f for f in demo_fc.columns if f not in ddCols])
#Rename Columns
db_demo.rename(columns={'PERMYEAR' : 'YEAR', 'ISSUED': 'ISSUEDATE'}, inplace=True)

In [ ]:
#Match the Columns to the existing Demo Permits Feature Class for use with the REST Server
dkCols = ['Name', 'TYPE', 'PERMSTATUS', 'DESCRIPTION', 'SHAPE']
rest_demo = demo_fc.drop(columns=[x for x in demo_fc.columns if x not in dkCols])

cols = {'TYPE': 'DemoType',
        'STATUS': 'Status',
        'DESCRIPTION': 'Descriptio'}
rest_demo.rename(columns=cols, inplace= True)

In [ ]:
demoRest = locGDB / f'Demo_Perms_REST_{dStr}'
demoEmp = locGDB / f'Demo_Perms_DB_{dStr}'
# de_fc = features.GeoAccessor.from_df(demo_fc, geometry_column='SHAPE')
# db_demo.spatial.to_featureclass(f'{demoEmp}', sanitize_columns=False)
rest_demo.spatial.to_featureclass(f'{demoRest}', sanitize_columns=False)

In [ ]:
# dupYearsF = []
# for field in range(len(dupYears)):
#     if int(dupYears[field]) > 80 and int(dupYears[field]) < 100:
#         dupYearsF.append("19" + str(field))
#     elif int(field) >= 0 and int(field) <=9:
#         dupYearsF.append("20" + str(field))
#     elif int(field) > 9 and int(field) < 80:
#          dupYearsF.append("20" + str(field))
# print(dupYearsF)

### Create Permits Feature Class for REST Server

<b> Export OWNERSHIPINFORMATION Table to local GDB. Change the path if setting environment variable. </b>

In [ ]:
owTable = wsDB + '\\PROD.gisadmin.OWNERSHIPINFORMATION'
owOutTable = f'OwnerTable_{dStr}'

arc.TableToTable_conversion(owTable, locGDB, owOutTable)

In [ ]:
# arc.ConvertTimeField_management use this when exporting the feature classes